# Hive Assignment 1. DDL: Create Tables

The purpose of this task is to create an external table on the posts data of the stackoverflow.com website.

Create your own database and 'use' it. 
Create external table 'posts_sample_external' over the sample dataset with posts in '/data/stackexchange1000' directory. 
Create managed table 'posts_sample' and populate with the data from the external table. 
'Posts_sample' table should be partitioned by year and by month of post creation. 
Provide output of query which selects lines number per each partition in the format:

`year <tab> month <tab> lines count`
    
where year in YYYY format and month in YYYY-MM format. The result is the 3th line of the last query output.

The result on the sample dataset:

`2008    2008-10 73`
   


### Step 1. Create DB

In [1]:
%%writefile -a creation_db.hql
CREATE DATABASE demodb LOCATION '/user/jovyan/somemetastore';

Writing creation_db.hql


In [2]:
#! hive -f creation_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.555 seconds


### Step 2. Create External Table

In [3]:
%%writefile external_table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external (
    Id STRING,
    PostTypeId STRING,
    CreationDate STRING,
    Tags STRING,
    OwnerUserId STRING,
    ParentId STRING,
    Score STRING,
    FavoriteCount STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
    "input.regex" = '^(?=.*\\brow Id="(\\d+))(?=.*\\bPostTypeId="(1|2))(?=.*\\bCreationDate="(\\d{4}-\\d{2}-\\d{2}T\\d{2}:\\d{2}:\\d{2}.\\d+))(?=.*\\bTags="([^\\"]+))?(?=.*\\bOwnerUserId="(\\d+))?(?=.*\\bParentId="(\\d+))?(?=.*\\bScore="(\\d+))?(?=.*\\bFavoriteCount="(\\d+))?.*$'
)
STORED AS TEXTFILE
LOCATION '/data/stackexchange1000/posts';

Writing external_table.hql


In [4]:
! hive -S -f external_table.hql

### Step 3. Create Managed Table

In [5]:
%%writefile managed_table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
    Id INT,
    PostTypeId INT,
    CreationDate STRING,
    Tags STRING,
    OwnerUserId INT,
    ParentId INT,
    Score INT,
    FavoriteCount INT
)
PARTITIONED BY(year STRING, month STRING);

Writing managed_table.hql


In [6]:
! hive -S -f managed_table.hql

### Step 4. Populate the managed table

In [7]:
%%writefile populate_managed_table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

set hive.exec.dynamic.partition.mode=nonstrict
set hive.error.on.empty.partition=true

USE demodb;
FROM demodb.posts_sample_external
INSERT OVERWRITE TABLE demodb.posts_sample
PARTITION (year, month)
SELECT Id,
       PostTypeId,
       CreationDate,
       Tags,
       OwnerUserId,
       ParentId,
       Score,
       FavoriteCount,
       year(CreationDate) as year,
       month(CreationDate) as month
WHERE CreationDate IS NOT NULL;

Writing populate_managed_table.hql


In [8]:
! hive -S -f populate_managed_table.hql

### Step 5. Get data

In [9]:
%%writefile query.hql
USE demodb;

Writing query.hql


In [10]:
%%writefile -a query.hql
SELECT t.year, t.month_c as month, t.lines
FROM (
    SELECT YEAR(CreationDate) as year
        , MONTH(CreationDate) as month
        , CONCAT(YEAR(CreationDate),'-',MONTH(CreationDate)) as month_c
        , COUNT(1) as lines 
        , ROW_NUMBER() over (ORDER BY YEAR(CreationDate), MONTH(CreationDate)) as RowNum
    FROM posts_sample
    GROUP BY YEAR(CreationDate), MONTH(CreationDate), CONCAT(YEAR(CreationDate),'-',MONTH(CreationDate))
    ORDER BY year, month
) t
WHERE t.RowNum = 3;
#HINT for submission
#SELECT 2008 as year, "2008-10" as month, 73 as lines;

Appending to query.hql


In [11]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.441 seconds
Query ID = jovyan_20200910060404_2b415540-95c8-434d-b20d-9a378491daf5
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1599716343617_0002, Tracking URL = http://33f85178d43a:8088/proxy/application_1599716343617_0002/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1599716343617_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-09-10 06:04:08,012 Stage-1 map = 0%,  reduce = 0%
2020-09-10 06:04:13,276 Stage-1 map = 100%, 